In [1]:
from models.model_utils import load_captioning_model
from preprocess.pipelinesV2 import build_pipeline, build_coco_pipeline
import tensorflow as tf
import re
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
import textwrap
import json
import pandas as pd
import os
import pickle
# Tokenizers
from tokenizers.tokenizers import KerasTokenizer
# Model
from models.CaptioningTransformer import CaptioningTransformer
from models.Callbacks import CaptioningCallback
# Results
from results.CaptioningFunctions import predict_caption, caption_dataset

In [2]:
model_name='20230126-060147'

config_path='../model_configs/'+model_name+'.config'
with open(config_path, 'rb') as fp:
    config = pickle.load(fp)


# Instantiate a model 
model=CaptioningTransformer(config)

# Obtain some training data
if config['img_backbone']=='efficientnetb0':
    img=np.random.randint(0,
                          256,
                          (1,*config['img_size']))
tokens=np.random.randint(0,
                         config['vocab_size'],
                         (1,config['capts_per_img'],config['seq_len']))
inputs=tf.data.Dataset.from_tensor_slices({'image':img,'tokens':tokens})

# Train the model for a single batch
loss=tf.keras.losses.SparseCategoricalCrossentropy(reduction='none',from_logits=False)
opt = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=opt, run_eagerly=True)
model.fit(inputs.batch(1))

# Load weights into model
model.load_weights('../models/'+model_name) 

2023-01-26 06:08:36.012916: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-26 06:08:36.013017: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-8-20221025-075908): /proc/driver/nvidia/version does not exist
2023-01-26 06:08:36.036765: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 8s 8s/step - loss: 28.1846 - acc: 0.0011
